In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import matplotlib.cm as cm
from sklearn.feature_selection import f_regression
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
data = pd.read_csv('../data/NBA_Players.csv')
data

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,PF,PTS,Potw,APG_Leader,MVP,PPG_Leader,RPG_Leader,Rookie,WS_Leader,Salary
0,1991,Alaa Abdelnaby,PF,22.0,POR,43,0,6.7,1.3,2.7,...,0.9,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,395000
1,1991,Mahmoud Abdul-Rauf,PG,21.0,DEN,67,19,22.5,6.2,15.1,...,2.2,14.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1660000
2,1991,Mark Acres,C,28.0,ORL,68,0,19.3,1.6,3.1,...,3.2,4.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,437000
3,1991,Michael Adams,PG,28.0,DEN,66,66,35.5,8.5,21.5,...,2.5,26.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,825000
4,1991,Mark Aguirre,SF,31.0,DET,78,13,25.7,5.4,11.7,...,2.7,14.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1115000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8998,2019,Delon Wright,PG,26.0,TOR,49,2,18.3,2.6,6.0,...,1.1,6.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2536898
8999,2019,Guerschon Yabusele,PF,23.0,BOS,41,1,6.1,0.9,1.9,...,0.8,2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2667600
9000,2019,Thaddeus Young,PF,30.0,IND,81,81,30.7,5.5,10.4,...,2.4,12.6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13764045
9001,2019,Trae Young,PG,20.0,ATL,81,81,30.9,6.5,15.5,...,1.7,19.1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5356440


Stats Dictionary:

Rk -- Rank
Pos -- Position
Age -- Age of Player at the start of February 1st of that season.
Tm -- Team
G -- Games
GS -- Games Started
MP -- Minutes Played Per Game
FG -- Field Goals Per Game
FGA -- Field Goal Attempts Per Game
FG% -- Field Goal Percentage
3P -- 3-Point Field Goals Per Game
3PA -- 3-Point Field Goal Attempts Per Game
3P% -- 3-Point Field Goal Percentage
2P -- 2-Point Field Goals Per Game
2PA -- 2-Point Field Goal Attempts Per Game
2P% -- 2-Point Field Goal Percentage
eFG% -- Effective Field Goal Percentage
This statistic adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.
FT -- Free Throws Per Game
FTA -- Free Throw Attempts Per Game
FT% -- Free Throw Percentage
ORB -- Offensive Rebounds Per Game
DRB -- Defensive Rebounds Per Game
TRB -- Total Rebounds Per Game
AST -- Assists Per Game
STL -- Steals Per Game
BLK -- Blocks Per Game
TOV -- Turnovers Per Game
PF -- Personal Fouls Per Game
PTS -- Points Per Game


In [3]:
position = pd.get_dummies(data['Pos'], drop_first= True)
position.columns = 'Pos_' + position.columns
data2 = pd.concat([data, position], axis=1).drop(columns = ['Pos','TRB'])
features_stats = data2.drop(columns = ['Year','Player','Tm','Potw','APG_Leader','MVP','PPG_Leader','RPG_Leader','Rookie','WS_Leader','Salary'])
Potw = data2['Potw']
position.head()

,Pos_PF,Pos_PG,Pos_SF,Pos_SG
0,1,0,0,0
1,0,1,0,0
2,0,0,0,0
3,0,1,0,0
4,0,0,1,0


## FULL MODEL

In [4]:
formula_1 = f'Potw ~ {" + ".join(list(features_stats.columns))}'
lm1 = smf.glm(formula_1, data = data2,family=sm.families.Binomial()).fit()
lm1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   Potw   No. Observations:                 9003
Model:                            GLM   Df Residuals:                     8973
Model Family:                Binomial   Df Model:                           29
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -796.00
Date:                Sat, 12 Oct 2019   Deviance:                       1592.0
Time:                        16:45:39   Pearson chi2:                 5.98e+03
No. Iterations:                    10                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -7.0533      2.202     -3.204      0.001     -11.368      -2.738
Age              0.0037      0.019      0.192      0.848      -0.034       0.041
G                0.0235      0.008      3.018      0.003       0.008       0.039
GS               0.0106      0.006      1.654      0.098      -0.002       0.023
MP              -0.0574      0.029     -1.996      0.046      -0.114      -0.001
FG               5.4060      2.123      2.546      0.011       1.245       9.567
FGA             -2.0274      1.347     -1.505      0.132      -4.667       0.612
FG_Prct         10.7732     11.362      0.948      0.343     -11.495      33.042
Three_P         -2.9811      1.901     -1.568      0.117      -6.707       0.745
Three_PA         1.9828      1.355      1.463      0.143      -0.673       4.639
Three_P_Prct    -0.7730      0.674     -1.147      0.252      -2.094       0.548
Two_P           -3.2803      1.530     -2.144      0.032      -6.279      -0.282
Two_PA           1.6402      1.353      1.212      0.225      -1.011       4.292
Two_P_Prct     -10.7823      4.834     -2.231      0.026     -20.256      -1.309
eFG_Prct        -4.6058      8.387     -0.549      0.583     -21.043      11.832
FT               1.1940      0.947      1.261      0.207      -0.662       3.050
FTA             -0.3099      0.276     -1.122      0.262      -0.851       0.231
FT_Prct         -0.9466      1.745     -0.542      0.588      -4.367       2.474
ORB             -0.0279      0.133     -0.209      0.834      -0.289       0.233
DRB              0.3841      0.067      5.692      0.000       0.252       0.516
AST              0.2203      0.065      3.375      0.001       0.092       0.348
STL              0.4287      0.192      2.229      0.026       0.052       0.806
BLK              0.5137      0.143      3.596      0.000       0.234       0.794
TOV             -0.0824      0.175     -0.470      0.638      -0.426       0.261
PF              -0.3444      0.147     -2.346      0.019      -0.632      -0.057
PTS             -0.3675      0.902     -0.407      0.684      -2.136       1.401
Pos_PF          -0.3807      0.258     -1.475      0.140      -0.887       0.125
Pos_PG           0.6144      0.467      1.316      0.188      -0.301       1.529
Pos_SF          -0.4112      0.355     -1.158      0.247      -1.107       0.285
Pos_SG           0.1630      0.406      0.401      0.688      -0.633       0.959
================================================================================
"""

## Use VIF to remove correlated predictors -- Multicollinearity

In [5]:
#Try to do VIF for the full model
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(features_stats.values, i) for i in range(features_stats.shape[1])]
vif["features"] = features_stats.columns
print(vif)

      VIF Factor      features
0      26.021094           Age
1      11.348225             G
2       6.761501            GS
3      83.415839            MP
4   17012.047220            FG
5   34424.254209           FGA
6     954.272979       FG_Prct
7     686.853563       Three_P
8    2536.168556      Three_PA
9       6.352874  Three_P_Prct
10   6836.728784         Two_P
11  22445.912054        Two_PA
12    169.022863    Two_P_Prct
13    785.512359      eFG_Prct
14    979.946294            FT
15    128.659018           FTA
16     25.767634       FT_Prct
17     11.994094           ORB
18     18.798239           DRB
19     12.483846           AST
20     10.229413           STL
21      3.989786           BLK
22     25.551175           TOV
23     20.839257            PF
24  21427.726312           PTS
25      2.349684        Pos_PF
26      4.973826        Pos_PG
27      3.154864        Pos_SF
28      4.056309        Pos_SG


In [6]:
# A function to remove a variable with max VIF for each VIF test, until all the remaining variables with VIF < 10.

def rm_var(df):
    while True:
        vif = pd.DataFrame()
        vif["VIF Factor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
        vif["features"] = df.columns
        max_vif_value = vif["VIF Factor"].max()
        if max_vif_value > 10:
            rm_var_name = vif.loc[vif["VIF Factor"] == max_vif_value,'features'].values[0]
            df.drop(columns=[rm_var_name], inplace = True)
            continue
        else:
            break
    return df.columns,vif
    

In [7]:
left_var = rm_var(features_stats)[0]
vif = rm_var(features_stats)[1]
print(vif)

    VIF Factor      features
0     7.950853             G
1     4.941416            GS
2     3.435240       Three_P
3     4.714193  Three_P_Prct
4     5.180555            FT
5     6.444955           ORB
6     8.287622           AST
7     9.341765           STL
8     3.264134           BLK
9     1.828716        Pos_PF
10    3.330801        Pos_PG
11    2.088936        Pos_SF
12    2.457646        Pos_SG


## REDUCED MODEL

In [8]:
# Remaining predictors
features_stats_2 = features_stats[left_var]
features_stats_2.head()

,G,GS,Three_P,Three_P_Prct,FT,ORB,AST,STL,BLK,Pos_PF,Pos_PG,Pos_SF,Pos_SG
0,43,0,0.0,0.000,0.6,0.6,0.3,0.1,0.3,1,0,0,0
1,67,19,0.4,0.240,1.3,0.5,3.1,0.8,0.1,0,1,0,0
2,68,0,0.0,0.333,1.0,2.1,0.4,0.4,0.4,0,0,0,0
3,66,66,2.5,0.296,7.0,0.9,10.5,2.2,0.1,0,1,0,0
4,78,13,0.3,0.308,3.1,1.7,1.8,0.6,0.3,0,0,1,0


In [9]:
formula_2 = f'Potw ~ {" + ".join(list(left_var))}'
lm2 = smf.glm(formula_2, data = data2,family=sm.families.Binomial()).fit()
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   Potw   No. Observations:                 9003
Model:                            GLM   Df Residuals:                     8989
Model Family:                Binomial   Df Model:                           13
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -898.51
Date:                Sat, 12 Oct 2019   Deviance:                       1797.0
Time:                        16:45:42   Pearson chi2:                 5.77e+03
No. Iterations:                     9                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -9.0605      0.510    -17.760      0.000     -10.060      -8.061
G                0.0078      0.008      1.038      0.299      -0.007       0.023
GS               0.0249      0.006      4.169      0.000       0.013       0.037
Three_P          0.6198      0.104      5.939      0.000       0.415       0.824
Three_P_Prct    -0.0596      0.599     -0.100      0.921      -1.233       1.114
FT               0.7744      0.045     17.153      0.000       0.686       0.863
ORB              0.2540      0.105      2.409      0.016       0.047       0.461
AST              0.3190      0.050      6.408      0.000       0.221       0.417
STL              0.3780      0.168      2.244      0.025       0.048       0.708
BLK              0.6229      0.122      5.119      0.000       0.384       0.861
Pos_PF          -0.3535      0.219     -1.616      0.106      -0.782       0.075
Pos_PG          -1.0769      0.373     -2.887      0.004      -1.808      -0.346
Pos_SF          -1.0006      0.288     -3.473      0.001      -1.565      -0.436
Pos_SG          -0.9127      0.311     -2.931      0.003      -1.523      -0.302
================================================================================
"""

## Deviance Test - First Time

In [29]:
delta_G = lm2.deviance - lm1.deviance
df_for_chi = len(lm1.params) - len(lm2.params)
# on significant level = 0.5, chi-square statistics
chi2_stat = stats.chi2.ppf(0.95, df_for_chi)
print(delta_G)
print(chi2_stat)
if delta_G > chi2_stat:
    print(f'{delta_G} > {chi2_stat}\nTherefore, we could reject null hypothesis on significant level of 0.05 and choose reduced model')

205.00795782822092
26.29622760486423
205.00795782822092 > 26.29622760486423
Therefore, we could reject null hypothesis on significant level of 0.05 and choose reduced model


## 